# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233472585558                   -0.50    7.0
  2   -7.249313271909       -1.80       -1.37    1.0
  3   -7.250211089174       -3.05       -1.60    4.0
  4   -7.250998642265       -3.10       -1.87    2.0
  5   -7.251312621238       -3.50       -2.42    2.0
  6   -7.251337391510       -4.61       -3.00    2.0
  7   -7.251338780970       -5.86       -3.93    3.0
  8   -7.251338795255       -7.85       -4.25    2.0
  9   -7.251338798284       -8.52       -4.71    1.0
 10   -7.251338798669       -9.42       -5.24    1.0
 11   -7.251338798703      -10.47       -5.98    2.0
 12   -7.251338798704      -11.83       -6.26    3.0
 13   -7.251338798705      -12.63       -6.87    3.0
 14   -7.251338798705      -14.10       -7.46    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.056176603526312735
[ Info: Arnoldi iteration step 2: normres = 0.5501724705911261
[ Info: Arnoldi iteration step 3: normres = 0.6885775396205283
[ Info: Arnoldi iteration step 4: normres = 0.24241353807532887
[ Info: Arnoldi iteration step 5: normres = 0.4486300283059171
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.95e-03, 7.79e-02, 4.03e-01, 1.81e-01, 1.69e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3188887858773606
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.71e-03, 1.36e-01, 2.52e-01, 1.06e-01, 2.17e-02)
[ Info: Arnoldi iteration step 7: normres = 0.1138445694924279
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.59e-04, 1.91e-02, 2.41e-02, 8.46e-02, 6.61e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09864292393329935
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.70e-